<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_ATH_TTM_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv')

mypf = mypf[mypf['LatestQ'] == 0]

stock_n100 = mypf['Symbol'].unique()

exclude = ['ACC','AMBUJACEM','CADILAHC','GILLETTE','NESTLEIND','PGHH','SIEMENS']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100
#stock_ids = ['DIVISLAB', 'HDFCBANK']
print('Stocks to be scanned:',len(stock_ids))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Stocks to be scanned: 71


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_latest_analysis = pd.DataFrame()

for stock_name in stock_ids: 


    print(stock_name)

    link_blueprint = "https://www.screener.in/company/"
    
    stock_list = ['AUBANK', 'COLPAL', 'GLAND','WHIRLPOOL']
    
    if stock_name in stock_list:
        actual_link = link_blueprint + stock_name.upper()
    else:
        actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"

    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')

        test_list = link_soup.find(id="profit-loss").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")        
        while(" \r" in test_list) :
            test_list.remove(" \r")

        return test_list

    test_list = get_list(actual_link)

    #print(len(test_list))
    n = len(test_list) 

    if n < 100:
        actual_link = link_blueprint + stock_name.upper() 
        test_list = get_list(actual_link)
        #print(len(test_list))

    q = 0

    if n > 162:
        p = 8
        q = n-162
    elif n > 120 and n <= 162:
        p = 8
    elif n < 120 and n > 100:
        p = 7   

    else:
        p = 6

    
    #print(test_list)
    if 'TTM' in test_list:
        end_year = 'TTM'
    else:
        end_year = 'Mar 2023'
        df_yoy = pd.DataFrame()      
        s = test_list.index(end_year) +1 - test_list.index('Mar 2017')

        df_yoy['Year'] = test_list[test_list.index('Mar 2017'):test_list.index(end_year)+1]
        df_yoy['Sales'] = test_list[test_list.index('Expenses\xa0+')-s:test_list.index('Expenses\xa0+')]
        df_yoy['Profit'] = test_list[test_list.index('Tax %')-s:test_list.index('Tax %')]

        df_yoy.set_index('Year', inplace=True)
        df_yoy = df_yoy.T
        df_yoy['Stock'] = stock_name.upper() + '.NS'
        #df_yoy
        df_latest_analysis = pd.concat([df_latest_analysis, df_yoy])
    
df_latest_analysis 

ADANIPORTS
ADANITRANS
APOLLOHOSP
ASIANPAINT
AUROPHARMA
BANDHANBNK
BANKBARODA
BATAINDIA
BERGEPAINT
BHARTIARTL
BIOCON
BOSCHLTD
BPCL
CANBK
CIPLA
COALINDIA
COLPAL
DEEPAKNTR
DIVISLAB
DLF
DMART
DRREDDY
EICHERMOT
EMAMILTD
GAIL
GLAND
GODREJCP
GODREJIND
GRASIM
HINDALCO
HINDPETRO
IBREALEST
IGL
INDIANB
INDIGO
INDIGOPNTS
IOC
IRCTC
ITC
JINDALSTEL
JSWSTEEL
JUBLFOOD
LT
LUPIN
M&M
MCDOWELL-N
MUTHOOTFIN
NAUKRI
NATIONALUM
NMDC
NTPC
ONGC
PAGEIND
PFIZER
PIDILITIND
PIIND
PNB
POWERGRID
PVR
SAIL
SBIN
SHREECEM
SUNPHARMA
SUZLON
TATAMOTORS
TIPSINDLTD
TORNTPHARM
UPL
VEDL
WHIRLPOOL
ZEEL


Year,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Stock
Sales,"78,164","85,244","99,586","96,080","90,026","109,714","138,252",COALINDIA.NS
Profit,"14,446","10,770","27,127","24,071","18,009","23,616","38,001",COALINDIA.NS


In [4]:
print('Stocks updated:', df_latest_analysis.shape[0])
if df_latest_analysis.shape[0] > 0:
  lastestq_stocks = list(df_latest_analysis['Stock'].unique())
  len(lastestq_stocks)

Stocks updated: 2


In [5]:
if df_latest_analysis.shape[0] > 0:
  #df_analysis.columns
  df_latest_analysis = df_latest_analysis.reset_index()
  df_latest_analysis.rename({'Mar 2023': 'Latest', 'index': 'Type'}, axis=1, inplace=True)
  cols = [ 'Mar 2017','Mar 2018', 'Mar 2019', 'Mar 2020', 'Mar 2021', 'Mar 2022', 'Latest']
  #df_analysis[cols] = df_analysis[cols].apply(pd.to_numeric)
  for col in cols:
      df_latest_analysis[col] = df_latest_analysis[col].str.replace(',', '').astype(float)

In [6]:
if df_latest_analysis.shape[0] > 0:
  df_latest_analysis['Max'] = df_latest_analysis.max(axis=1)
  df_latest_analysis['Diff%'] = round((df_latest_analysis['Max'] - df_latest_analysis['Latest'])*100/df_latest_analysis['Max'],2)

<ipython-input-6-ff4f08accfa3>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_latest_analysis['Max'] = df_latest_analysis.max(axis=1)


In [7]:
if df_latest_analysis.shape[0] > 0:
  df_master_analysis = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100-ATH-TTM.csv')
  df_master_analysis.columns = df_latest_analysis.columns
  df_master_analysis = df_master_analysis[~df_master_analysis['Stock'].isin(lastestq_stocks)]
  df_master_analysis = pd.concat([df_master_analysis, df_latest_analysis])
  df_master_analysis.to_csv('/content/drive/My Drive/data/stocks/Nifty100-ATH-TTM.csv', index=False)

In [8]:
if df_latest_analysis.shape[0] > 0:
  df_tmp = df_latest_analysis[df_latest_analysis['Diff%'] <= 5]
  tmp = pd.DataFrame(df_tmp['Stock'].value_counts())
  len(tmp[tmp['Stock']>1].index)
  star_stocks = list(tmp[tmp['Stock']>1].index)
  len(star_stocks)

In [9]:
mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv')
mypf.loc[mypf['Stock'].isin(star_stocks), 'StarStock'] = 1
mypf.loc[mypf['Stock'].isin(lastestq_stocks), 'LatestQ'] = 1
mypf.to_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv', index=False)